# 13 - Text Similarity
by [Alejandro Correa Bahnsen](http://www.albahnsen.com/) 

version 1.0, June 2020

## Part of the class [AdvancedMethodsDataAnalysisClass](https://github.com/albahnsen/AdvancedMethodsDataAnalysisClass/tree/master/notebooks)

This notebook is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). Special thanks goes to [
Adrien sieg](https://medium.com/@adriensieg/text-similarities-da019229c894)

## Text Similarities : Estimate the degree of similarity between two texts

We always need to compute the similarity in meaning between texts.
* Search engines need to model the relevance of a document to a query, beyond the overlap in words between the two. For instance, question-and-answer sites such as Quora or Stackoverflow need to determine whether a question has already been asked before.
* In legal matters, text similarity task allow to mitigate risks on a new contract, based on the assumption that if a new contract is similar to a existent one that has been proved to be resilient, the risk of this new contract being the cause of financial loss is minimised. Here is the principle of Case Law principle. Automatic linking of related documents ensures that identical situations are treated similarly in every case. Text similarity foster fairness and equality. Precedence retrieval of legal documents is an information retrieval task to retrieve prior case documents that are related to a given case document.
* In customer services, AI system should be able to understand semantically similar queries from users and provide a uniform response. The emphasis on semantic similarity aims to create a system that recognizes language and word patterns to craft responses that are similar to how a human conversation works. For example, if the user asks “What has happened to my delivery?” or “What is wrong with my shipping?”, the user will expect the same response.

### What is text similarity?
Text similarity has to determine how ‘close’ two pieces of text are both in surface closeness [lexical similarity] and meaning [semantic similarity].

# 1. Jaccard Similarity

Jaccard similarity or intersection over union is defined as size of intersection divided by size of union of two sets. Let’s take example of two sentences:
- Sentence 1: AI is our friend and it has been friendly
- Sentence 2: AI and humans have always been friendly

<img src="ts1.png"
     style="margin-right: 10px;" />

In order to calculate similarity using Jaccard similarity, we will first perform lemmatization to reduce words to the same root word. In our case, “friend” and “friendly” will both become “friend”, “has” and “have” will both become “has”.


In [ ]:
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [ ]:
s1 = "AI is our friend and it has been friendly"
s2 = "AI and humans have always been friendly"

In [ ]:
jaccard_similarity(s1, s2)

# 2. Cosine Similarity

Cosine similarity calculates similarity by measuring the cosine of angle between two vectors.


<img src="ts2.png"
     style="margin-right: 10px;" />
     
     
Mathematically speaking, Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0,π] radians. It is thus a judgment of orientation and not magnitude: two vectors with the same orientation have a cosine similarity of 1, two vectors oriented at 90° relative to each other have a similarity of 0, and two vectors diametrically opposed have a similarity of -1, independent of their magnitude.

The cosine similarity is advantageous because even if the two similar documents are far apart by the Euclidean distance (due to the size of the document), chances are they may still be oriented closer together. The smaller the angle, higher the cosine similarity.


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
import numpy as np

def cosine_distance_countVectorizer(s1, s2):

    vect = CountVectorizer()
    X_dtm = vect.fit_transform([s1, s2]).todense()
    
    return 1-cosine(X_dtm[0], X_dtm[1])

In [5]:
cosine_distance_countVectorizer(s1, s2)

0.5039526306789696

# 3. Sentence Encoding + Cosine Similarity

Word embedding is one of the most popular representation of document vocabulary. It is capable of capturing context of a word in a document, semantic and syntactic similarity, relation with other words, etc.

It is common to find in many sources that the first step to cluster text data is to transform text units to vectors. This is not 100% true. But this step depends mostly on the similarity measure and the clustering algorithm. Some of the best performing text similarity measures don’t use vectors at all. This is the case of the winner system in SemEval2014 sentence similarity task which uses lexical word alignment. However, vectors are more efficient to process and allow to benefit from existing ML/DL algorithms.


<img src="ts3.png"
     style="margin-right: 10px;" />
     

In [6]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf
#To make tf 2.0 compatible with tf1.0 code, we disable the tf2.0 functionalities
tf.disable_eager_execution()
import tensorflow_hub as hub

In [7]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

In [8]:
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentences_embeddings = session.run(embed([s1, s2]))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [9]:
sentences_embeddings

array([[-0.01817411, -0.01120597, -0.03367907, ..., -0.03023395,
        -0.06816524, -0.00829004],
       [-0.05866046, -0.03078497, -0.0389335 , ..., -0.04161206,
        -0.01894294, -0.06467941]], dtype=float32)

In [10]:
sentences_embeddings.shape

(2, 512)

In [11]:
1-cosine(sentences_embeddings[0], sentences_embeddings[1])

0.7773129343986511